In [1]:
import pandas as pd
import numpy as np 
import torch
from torch import nn
from torch.utils import data
from torch.nn import functional as F
from torchsummary import summary
from torch.autograd import Variable

ImportError: Failed to load PyTorch C extensions:
    It appears that PyTorch has loaded the `torch/_C` folder
    of the PyTorch repository rather than the C extensions which
    are expected in the `torch._C` namespace. This can occur when
    using the `install` workflow. e.g.
        $ python setup.py install && python -c "import torch"

    This error can generally be solved using the `develop` workflow
        $ python setup.py develop && python -c "import torch"  # This should succeed
    or by running Python from a different directory.

In [454]:
print('GPU:', torch.cuda.is_available())

GPU: True


# Load Data

In [455]:
# Read the data
import glob,os
path=r'../Dataset/hai-master/hai-21.03'
file=glob.glob(os.path.join(path, "test1.csv"))
print(file)
dl= []
for f in file:
 dl.append(pd.read_csv(f,header=[0],index_col=None))
test_df=pd.concat(dl)

['../Dataset/hai-master/hai-21.03\\test1.csv']


In [456]:
# Read the data
import glob,os
path=r'../Dataset/hai-master/hai-21.03'
file=glob.glob(os.path.join(path, "test2.csv"))
print(file)
dl= []
for f in file:
 dl.append(pd.read_csv(f,header=[0],index_col=None))
test2_df=pd.concat(dl)

['../Dataset/hai-master/hai-21.03\\test2.csv']


In [457]:
# # Read the data
# import glob,os
# path=r'../Dataset/hai-master/hai-21.03'
# file=glob.glob(os.path.join(path, "train*.csv"))
# print(file)
# dl= []
# for f in file:
#  dl.append(pd.read_csv(f,header=[0],index_col=None))
# train_df=pd.concat(dl)

In [458]:
# dl= []

In [459]:
# dl.append(pd.read_csv('../Dataset\\data1.csv',header=[0],index_col=None))
# dl.append(pd.read_csv('../Dataset\\data2.csv',header=[0],index_col=None))
# df=pd.concat(dl)

In [460]:
test_df.shape

(43201, 84)

In [461]:
test_df.head()

,time,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,...,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
0,2020-07-07 15:00:00,0.10178,1.58771,403.78854,985.37353,32.59527,100.0,2839.58520,36.81010,100.0,...,20469.0,386.26666,380.31683,0,10044.0,27567.0,0,0,0,0
1,2020-07-07 15:00:01,0.10178,1.58725,403.78854,985.37353,32.59527,100.0,2843.37549,36.80895,100.0,...,20489.0,386.30286,380.02747,0,10040.0,27564.0,0,0,0,0
2,2020-07-07 15:00:02,0.10178,1.59519,403.78854,985.37353,32.59527,100.0,2845.06006,36.82879,100.0,...,20604.0,389.73883,381.52850,0,10037.0,27565.0,0,0,0,0
3,2020-07-07 15:00:03,0.10178,1.59747,403.78854,985.37353,32.59527,100.0,2837.33911,36.83451,100.0,...,20633.0,388.94311,382.08911,0,10040.0,27560.0,0,0,0,0
4,2020-07-07 15:00:04,0.10178,1.59869,403.78854,985.37353,32.59527,100.0,2843.37549,36.83756,100.0,...,20738.0,389.72082,383.44543,0,10042.0,27564.0,0,0,0,0


In [462]:
test_df['attack'].value_counts()

0    42572
1      629
Name: attack, dtype: int64

In [463]:
test_df

,time,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,...,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
0,2020-07-07 15:00:00,0.10178,1.58771,403.78854,985.37353,32.59527,100.00000,2839.58520,36.81010,100.00000,...,20469.0,386.26666,380.31683,0,10044.0,27567.0,0,0,0,0
1,2020-07-07 15:00:01,0.10178,1.58725,403.78854,985.37353,32.59527,100.00000,2843.37549,36.80895,100.00000,...,20489.0,386.30286,380.02747,0,10040.0,27564.0,0,0,0,0
2,2020-07-07 15:00:02,0.10178,1.59519,403.78854,985.37353,32.59527,100.00000,2845.06006,36.82879,100.00000,...,20604.0,389.73883,381.52850,0,10037.0,27565.0,0,0,0,0
3,2020-07-07 15:00:03,0.10178,1.59747,403.78854,985.37353,32.59527,100.00000,2837.33911,36.83451,100.00000,...,20633.0,388.94311,382.08911,0,10040.0,27560.0,0,0,0,0
4,2020-07-07 15:00:04,0.10178,1.59869,403.78854,985.37353,32.59527,100.00000,2843.37549,36.83756,100.00000,...,20738.0,389.72082,383.44543,0,10042.0,27564.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43196,2020-07-08 02:59:56,0.10109,1.61632,395.43826,1115.26257,31.81634,79.42770,2440.14233,36.10442,37.51368,...,21044.0,394.91107,390.13672,50,10053.0,27593.0,0,0,0,0
43197,2020-07-08 02:59:57,0.10109,1.61891,395.43826,1115.26257,31.81634,79.52109,2455.06909,36.11090,37.72263,...,21073.0,397.35248,390.24518,50,10053.0,27599.0,0,0,0,0
43198,2020-07-08 02:59:58,0.10109,1.61815,395.43826,1115.26257,31.81634,79.62234,2446.70361,36.10899,37.83055,...,21064.0,396.70142,390.46222,50,10052.0,27599.0,0,0,0,0
43199,2020-07-08 02:59:59,0.10109,1.62609,395.43826,1115.26257,31.81634,79.71570,2444.73511,36.12883,37.67564,...,21171.0,397.73218,391.78241,50,10053.0,27603.0,0,0,0,0


In [464]:
test_df.describe()

,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,P1_FCV01Z,...,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
count,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,...,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.000000,43201.0
mean,0.101221,1.396985,398.987740,1074.705235,32.317316,63.082296,1885.537917,36.056749,56.355013,56.188993,...,17887.158266,331.511471,331.787760,20.692808,10049.854488,27609.174562,0.014560,0.011111,0.003449,0.0
std,0.000484,0.127081,5.104598,46.375027,0.455703,40.130593,1152.533311,0.556682,42.174724,42.144635,...,1755.223122,36.113908,32.548281,24.626452,19.543202,24.112590,0.119784,0.104822,0.058627,0.0
min,0.100490,1.049520,394.214750,985.373530,31.816340,0.000000,28.005140,34.703660,0.000000,0.268550,...,13075.000000,239.854550,243.001280,0.000000,9807.000000,27539.000000,0.000000,0.000000,0.000000,0.0
25%,0.101090,1.322890,395.438260,1057.218380,31.816340,22.081260,712.246030,35.601640,14.917670,14.459230,...,16863.000000,306.731020,312.789370,0.000000,10051.000000,27596.000000,0.000000,0.000000,0.000000,0.0
50%,0.101090,1.396450,395.438260,1086.267580,32.431240,81.730390,2793.821050,36.092560,57.179540,57.260130,...,17881.000000,332.157840,331.741880,0.000000,10053.000000,27625.000000,0.000000,0.000000,0.000000,0.0
75%,0.101770,1.470140,403.788540,1115.262570,32.595270,100.000000,2836.356440,36.498820,100.000000,99.794000,...,18896.000000,354.437930,350.513550,50.000000,10053.000000,27627.000000,0.000000,0.000000,0.000000,0.0
max,0.101780,1.963000,406.612850,1115.262570,32.963250,100.000000,2860.185300,37.492030,100.000000,99.992370,...,25709.000000,488.805720,476.743350,50.000000,10329.000000,27627.000000,1.000000,1.000000,1.000000,0.0


In [465]:
# np.any(np.isnan(df))

In [466]:
# torch.cuda.is_available()

Data preprocessing

In [467]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
def data_preprocessing(test_df):
    y = test_df.attack
    x = test_df.drop(['attack','attack_P1','attack_P2','attack_P3','time'], axis=1)
    values = x
    # np.all(np.isfinite(values))
    values.replace([np.inf, -np.inf], np.nan, inplace=True)
    #normalization
    scaler = MinMaxScaler()
    X_normalized=scaler.fit_transform(values)
    X_normalized
    from sklearn.model_selection import train_test_split
    # # Divide data into training and validation subsets
    X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_normalized, y, train_size=0.8, test_size=0.2,
                                                                random_state=0, shuffle=False)
    return X_train_full, X_valid_full, y_train, y_valid

In [468]:
# label Encoding
# df['marker'] = df['marker'].replace(['Natural'],0)
# df['marker'] = df['marker'].replace(['Attack'],1)
# df['marker'].value_counts()

In [469]:
dataset = test_df.iloc[:]
missing_val_count_by_column = (dataset.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

Series([], dtype: int64)


In [470]:
dataset2 = test2_df.iloc[:]

In [471]:
# #normalization
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler(feature_range=(-1,1))
# df_normalized=scaler.fit_transform(dataset.values.reshape(-1,1))

In [472]:
# Separate target from predictors
y = test_df.attack
test_x = test_df.drop(['attack','attack_P1','attack_P2','attack_P3','time'], axis=1)

In [473]:
y_2 = test2_df.attack
test2_x = test2_df.drop(['attack','attack_P1','attack_P2','attack_P3','time'], axis=1)

In [474]:
print(y_2.shape)
print(test2_x .shape)

(118801,)
(118801, 79)


In [475]:
print(y.shape)
print(test_x.shape)

(43201,)
(43201, 79)


In [476]:
values = test_x
# np.all(np.isfinite(values))
values.replace([np.inf, -np.inf], np.nan, inplace=True)


In [477]:
#normalization
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_normalized=scaler.fit_transform(values)
X_normalized

array([[1.        , 0.58916451, 0.77219816, ..., 0.        , 0.45402299,
        0.31818182],
       [1.        , 0.58866094, 0.77219816, ..., 0.        , 0.44636015,
        0.28409091],
       [1.        , 0.59735298, 0.77219816, ..., 0.        , 0.44061303,
        0.29545455],
       ...,
       [0.46511628, 0.62248763, 0.09868528, ..., 1.        , 0.46934866,
        0.68181818],
       [0.46511628, 0.63117966, 0.09868528, ..., 1.        , 0.47126437,
        0.72727273],
       [0.46511628, 0.63634672, 0.09868528, ..., 1.        , 0.47126437,
        0.63636364]])

In [478]:
from sklearn.model_selection import train_test_split
# # Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X_normalized, y, train_size=0.5, test_size=0.3,
                                                                random_state=0, shuffle=False)

In [479]:
X_train_full.shape

(21600, 79)

In [480]:
# from keras.preprocessing.sequence import TimeseriesGenerator

In [481]:
X_train_full

array([[1.        , 0.58916451, 0.77219816, ..., 0.        , 0.45402299,
        0.31818182],
       [1.        , 0.58866094, 0.77219816, ..., 0.        , 0.44636015,
        0.28409091],
       [1.        , 0.59735298, 0.77219816, ..., 0.        , 0.44061303,
        0.29545455],
       ...,
       [0.99224806, 0.37734816, 0.        , ..., 0.        , 0.49425287,
        0.80681818],
       [0.99224806, 0.38887551, 0.        , ..., 0.        , 0.47126437,
        0.75      ],
       [0.99224806, 0.38786837, 0.        , ..., 0.        , 0.47126437,
        0.84090909]])

In [482]:
y_train

0        0
1        0
2        0
3        0
4        0
        ..
21595    0
21596    0
21597    0
21598    0
21599    0
Name: attack, Length: 21600, dtype: int64

In [483]:
#function to convert to time domain dataset
def create_dataset(X, time_steps):
    Xs = []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
    return np.array(Xs)

In [484]:
# 79 + 1 column

def create_Used_Dataset(X):
    label_column = np.ones((len(X),1), dtype = np.object)
    # used = 1
    label_column[:] = "used"
    print(X.shape,label_column.shape)
    labeled_X = np.c_[X[:,0],label_column]
    return labeled_X


 60  3351
 x * 60

In [485]:
def data_Selection(used_dataset):  
    
    return used_dataset


used=1 mature=2 preserve=3 Discard=4

In [486]:
# X_train = pd.DataFrame(X_train_full)
# test_x, v =  create_dataset(X_train, 1)
# test_x.shape

In [487]:
# labeled_X = create_Used_Dataset(test_x)

In [488]:
# labeled_X

In [489]:
# labeled_X.shape

In [490]:
# def data_Selection(used_dataset):


In [491]:
array = np.empty((4019,1))

In [492]:
array[:] = 1

In [493]:
# reshape to [samples, time_steps, n_features]

TIME_STEPS = 1
X_train = pd.DataFrame(X_train_full)
X_train = create_dataset(X_train, TIME_STEPS)
y_train = pd.DataFrame(y_train)
y_train = create_dataset(y_train, TIME_STEPS)
X_valid = pd.DataFrame(X_valid_full)
X_valid = create_dataset(X_valid, TIME_STEPS)
y_valid = pd.DataFrame(y_valid)
y_valid = create_dataset(y_valid, TIME_STEPS)
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)


(21599, 1, 79)
(21599, 1, 1)
(12960, 1, 79)
(12960, 1, 1)


In [494]:
X_train[1]

array([[1.00000000e+00, 5.88660945e-01, 7.72198159e-01, 0.00000000e+00,
        6.79155296e-01, 1.00000000e+00, 9.94064710e-01, 7.55025337e-01,
        1.00000000e+00, 9.99234987e-01, 0.00000000e+00, 3.07852758e-04,
        3.04379107e-01, 3.22222285e-01, 2.52870604e-01, 4.87429581e-01,
        9.88299120e-01, 9.94659506e-01, 5.23631731e-01, 5.90182566e-01,
        2.80442318e-01, 2.73194067e-01, 1.62877630e-01, 2.95910648e-01,
        3.00209376e-01, 0.00000000e+00, 1.63552175e-01, 4.63048094e-01,
        4.80859032e-02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        7.53440645e-02, 3.09865863e-02, 3.56638155e-01, 0.00000000e+00,
        0.00000000e+00, 6.84379001e-01, 0.00000000e+00, 4.31492048e-01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        7.05084746e-01, 7.05506333e-01, 0.00000000e+00, 7.22790861e-01,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.000000

In [496]:
# reshape input to be 3D [samples, timesteps, features]
# X_train_full = X_train_full.reshape((X_train_full.shape[0], 1, X_train_full.shape[1]))
# X_valid_full = X_valid_full.reshape((X_valid_full.shape[0], 1, X_valid_full.shape[1]))
# print(X_train_full.shape, y_train.shape, X_valid_full.shape, y_valid.shape) 

# Construct Model

In [497]:
# class LSTM(nn.Module):
#     def __init__(self,input_size, output_size, hidden_size, num_layers):
#         super(LSTM,self).__init__()
#         self.input_size = input_size
#         self.output_size = output_size
#         self.num_layers = num_layers
#         self.hidden_size = hidden_size
        
        

       
#         self.lstm1 = nn.LSTM(input_size=input_size, 
#                             hidden_size=hidden_size,
#                             num_layers=num_layers, 
#                             batch_first=False,dropout = 0)
#         self.lstm2 = nn.LSTM(input_size=hidden_size, 
#                             hidden_size=32,
#                             num_layers=num_layers, 
#                             batch_first=False,dropout = 0)
#         self.dropout1 = nn.Dropout(p=0.3)
#         self.dropout2 = nn.Dropout(p=0.3)
#         self.fc = nn.Linear(32,output_size)
      
#         self.sigmoid = nn.Sigmoid()
     
#     def forward(self, x):   
#         per_out=[]
#         lstm_out, self.hidden_cell = self.lstm1(x)
#         per_out.append(lstm_out)
#         lstm_out = self.dropout1(lstm_out)
#         lstm_out, self.hidden_cell = self.lstm2(lstm_out)
#         per_out.append(lstm_out)
#         lstm_out = self.dropout2(lstm_out)
#         out = self.fc(lstm_out)
#         per_out.append(out)
#         score = self.sigmoid(out)
#         return score, per_out

In [498]:
class LSTM(nn.Module):
    def __init__(self,input_size, output_size, hidden_size, num_layers):
        super(LSTM,self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        #layer 1
        self.lstm = nn.LSTM(self.input_size, 
                            self.hidden_size,
                            self.num_layers,
                            batch_first=False,dropout = 0)

        self.dropout = nn.Dropout(p=0.3)
        #layer 2
        self.fc = nn.Linear(hidden_size,output_size)
      
        self.sigmoid = nn.Sigmoid()
     
    def forward(self, x):     
        per_out=[]
        lstm_out, self.hidden_cell = self.lstm(x)
        per_out.append(lstm_out)
        lstm_out = self.dropout(lstm_out)    
        out = self.fc(lstm_out)
        per_out.append(out)
        score = self.sigmoid(out)
        return score, per_out

In [499]:
def evaluate_accuracy(x,y,model):
    output,pre_out = model(x)
    output = torch.reshape(output,[-1,1])
    correct = (output.ge(0.5) == y).sum().item()
    n = y.shape[0]
    return correct/n

In [500]:
input_size = 79
output_size = 1
hidden_size = 20
num_layers =2

device = torch.device('cuda')
model = LSTM(input_size,output_size, hidden_size, num_layers).to(device)
loss_function = nn.BCELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
# optimizer = torch.optim.SGD(model.parameters(),lr=0.01)

In [501]:
def getDataloader(X_train_full,y_train,X_valid_full,y_valid):
    TIME_STEPS = 1
    X_train = pd.DataFrame(X_train_full)
    X_train = create_dataset(X_train, TIME_STEPS)
    y_train = pd.DataFrame(y_train)
    y_train = create_dataset(y_train, TIME_STEPS)
    X_valid = pd.DataFrame(X_valid_full)
    X_valid = create_dataset(X_valid, TIME_STEPS)
    y_valid = pd.DataFrame(y_valid)
    y_valid = create_dataset(y_valid, TIME_STEPS)
# 将输入和输出封装进Data.TensorDataset()类对象
    x = torch.tensor(X_train[:,:,:]).float()
    x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    y = y_train[:]
    y = np.array(y)
# y = torch.tensor(np.reshape(y,[-1,1]))
    y = torch.tensor(y)
    y = y.float()
    torch_dataset = Data.TensorDataset(x,y)
    train_dataloader = torch.utils.data.DataLoader(torch_dataset, batch_size=60, shuffle=False, num_workers=0)
    return X_train, y_train, X_valid, y_valid, train_dataloader

In [502]:
import torch.utils.data as Data
# 将输入和输出封装进Data.TensorDataset()类对象
x = torch.tensor(X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
# y = torch.tensor(np.reshape(y,[-1,1]))
y = torch.tensor(y)
y = y.float()
print(x.shape)
print(y.shape)

torch_dataset = Data.TensorDataset(x,y)

torch.Size([21599, 1, 79])
torch.Size([21599, 1, 1])


In [503]:
BATCH_SIZE = 60
train_dataloader = torch.utils.data.DataLoader(torch_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [504]:
# train_dataloader = torch.utils.data.DataLoader(train_datasets, batch_size=batch_size, shuffle=True, num_workers=2)

# Training and Evaluation

In [505]:
train_dataloader.batch_size

60

In [506]:
len(train_dataloader)

360

In [507]:
batch_array = np.zeros(len(train_dataloader))

In [508]:
batch_index_array = np.zeros(len(train_dataloader))

In [509]:
len(X_train)

21599

In [510]:
import time

time_start=time.time()

epochs = 1

for epoch in range(epochs):
    model.train()
    for batch, (batch_x, batch_y) in enumerate(train_dataloader):   
        batch_x = batch_x.cuda()   
        output,pre_out = model(batch_x)
        output = torch.reshape(output,[-1,1])

        batch_y = np.array(batch_y)
        batch_y = torch.tensor(np.reshape(batch_y,[-1,1]))
        batch_y = batch_y.float()
        batch_y = batch_y.cuda()

        loss = loss_function(output,batch_y)
        acc = evaluate_accuracy(batch_x,batch_y,model)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_array[batch] = loss.item()
        batch_index_array[batch] = batch
        if epoch % 5 == 0:
            print("epoch:{} batch:{} loss:{} acc:{}".format(epoch,batch,loss.item(),acc))

time_end=time.time()
print('totally cost',time_end-time_start)

epoch:0 batch:0 loss:0.7773289084434509 acc:0.0
epoch:0 batch:1 loss:0.7592916488647461 acc:0.0
epoch:0 batch:2 loss:0.7515842318534851 acc:0.0
epoch:0 batch:3 loss:0.741840124130249 acc:0.016666666666666666
epoch:0 batch:4 loss:0.7339040040969849 acc:0.05
epoch:0 batch:5 loss:0.7180477380752563 acc:0.23333333333333334
epoch:0 batch:6 loss:0.7060229778289795 acc:0.3
epoch:0 batch:7 loss:0.7004945278167725 acc:0.6
epoch:0 batch:8 loss:0.6854856610298157 acc:0.6
epoch:0 batch:9 loss:0.6676378846168518 acc:0.8
epoch:0 batch:10 loss:0.6454855799674988 acc:0.8833333333333333
epoch:0 batch:11 loss:0.6444611549377441 acc:0.8833333333333333
epoch:0 batch:12 loss:0.629086971282959 acc:0.9333333333333333
epoch:0 batch:13 loss:0.6129032373428345 acc:0.9666666666666667
epoch:0 batch:14 loss:0.6000858545303345 acc:0.9333333333333333
epoch:0 batch:15 loss:0.5832686424255371 acc:0.95
epoch:0 batch:16 loss:0.5586196184158325 acc:0.9666666666666667
epoch:0 batch:17 loss:0.5464609265327454 acc:0.9833333

epoch:0 batch:167 loss:0.04683859646320343 acc:1.0
epoch:0 batch:168 loss:0.04894496873021126 acc:1.0
epoch:0 batch:169 loss:0.05322088673710823 acc:1.0
epoch:0 batch:170 loss:0.04140046611428261 acc:1.0
epoch:0 batch:171 loss:0.04383455961942673 acc:1.0
epoch:0 batch:172 loss:0.04799081012606621 acc:1.0
epoch:0 batch:173 loss:0.04424075409770012 acc:1.0
epoch:0 batch:174 loss:0.04603787511587143 acc:1.0
epoch:0 batch:175 loss:0.04727259278297424 acc:1.0
epoch:0 batch:176 loss:0.04775870963931084 acc:1.0
epoch:0 batch:177 loss:0.04695666953921318 acc:1.0
epoch:0 batch:178 loss:0.04362800344824791 acc:1.0
epoch:0 batch:179 loss:0.044211193919181824 acc:1.0
epoch:0 batch:180 loss:0.04594562202692032 acc:1.0
epoch:0 batch:181 loss:0.04210630804300308 acc:1.0
epoch:0 batch:182 loss:0.04743356257677078 acc:1.0
epoch:0 batch:183 loss:0.04348258301615715 acc:1.0
epoch:0 batch:184 loss:0.04326717182993889 acc:1.0
epoch:0 batch:185 loss:0.04420000687241554 acc:1.0
epoch:0 batch:186 loss:0.04355

epoch:0 batch:336 loss:0.03361717239022255 acc:1.0
epoch:0 batch:337 loss:0.03076714649796486 acc:1.0
epoch:0 batch:338 loss:0.036388300359249115 acc:1.0
epoch:0 batch:339 loss:0.03427756950259209 acc:1.0
epoch:0 batch:340 loss:0.031628966331481934 acc:1.0
epoch:0 batch:341 loss:0.03596169501543045 acc:1.0
epoch:0 batch:342 loss:0.03249558433890343 acc:1.0
epoch:0 batch:343 loss:0.03177730739116669 acc:1.0
epoch:0 batch:344 loss:0.037233978509902954 acc:1.0
epoch:0 batch:345 loss:0.03610067442059517 acc:1.0
epoch:0 batch:346 loss:0.03614571690559387 acc:1.0
epoch:0 batch:347 loss:0.034504830837249756 acc:1.0
epoch:0 batch:348 loss:0.0322682149708271 acc:1.0
epoch:0 batch:349 loss:0.03587501868605614 acc:1.0
epoch:0 batch:350 loss:0.030219299718737602 acc:1.0
epoch:0 batch:351 loss:0.03488939628005028 acc:1.0
epoch:0 batch:352 loss:0.035641323775053024 acc:1.0
epoch:0 batch:353 loss:0.031193025410175323 acc:1.0
epoch:0 batch:354 loss:0.03684031218290329 acc:1.0
epoch:0 batch:355 loss:0.

In [511]:
model.eval()
x = torch.tensor( X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
y = torch.tensor(np.reshape(y,[-1,1]))
y = y.float()
    
x = x.cuda()
y = y.cuda()

x_valid = torch.tensor(X_valid[:,:,:]).float()
x_valid = torch.where(torch.isnan(x_valid), torch.full_like(x_valid, 0), x_valid)
    
y_val = y_valid[:]
y_val = np.array(y_val)
y_val = torch.tensor(np.reshape(y_val,[-1,1]))
y_val = y_val.float()

x_valid = x_valid.cuda()
y_val = y_val.cuda()

print("epoch:{} loss:{}".format(epoch,loss.item()))
print("train acc", evaluate_accuracy(x,y,model))
print("valid acc:",evaluate_accuracy(x_valid,y_val,model))

epoch:0 loss:0.033212676644325256
train acc 0.9749988425390065
valid acc: 0.9931327160493827


In [512]:
batch_array

array([0.77732891, 0.75929165, 0.75158423, 0.74184012, 0.733904  ,
       0.71804774, 0.70602298, 0.70049453, 0.68548566, 0.66763788,
       0.64548558, 0.64446115, 0.62908697, 0.61290324, 0.60008585,
       0.58326864, 0.55861962, 0.54646093, 0.52154589, 0.51167285,
       0.49475542, 0.48857272, 0.45923901, 0.46569294, 0.43180919,
       0.4126296 , 0.40010762, 0.39108792, 0.37340677, 0.37078795,
       0.35216829, 0.34579664, 0.31592327, 0.31547108, 0.30624518,
       1.27206385, 1.44977665, 1.48180604, 0.7159363 , 0.27935183,
       0.27320725, 0.28638431, 0.28408587, 0.278124  , 0.29335231,
       0.27344793, 0.26695177, 0.26871523, 0.2475822 , 0.25156337,
       0.25561041, 0.24119559, 0.24650316, 0.23098692, 0.22968651,
       0.22321308, 0.21860774, 0.22402176, 0.21720611, 0.21417044,
       0.20073754, 0.21739592, 0.18915933, 0.1763349 , 0.17590524,
       0.18545066, 0.18260938, 0.18271153, 0.16131654, 0.16486415,
       0.16205582, 0.15758161, 0.14810373, 0.15076056, 0.15595

In [513]:
# 1，datafram和array的相互转换
# dataframe转化成array
# df=df.values

# array转化成dataframe

# import pandas as pd

# df = pd.DataFrame(df)
# df1 = df1[(df1['网元ID'] == netid) & (df1['cellId'] == cid)]
# df2 = df2[(df2['网元ID'] == netid) & (df2['cellId'] == cid)]


In [520]:
batch_df = pd.DataFrame(batch_array)
batch_df

,0
0,0.777329
1,0.759292
2,0.751584
3,0.741840
4,0.733904
...,...
355,0.029425
356,0.032624
357,0.033729
358,0.032058


In [521]:
wasted_df = batch_df[batch_df[0] >= 1]
wasted_array = wasted_df.index
wasted_array

Int64Index([35, 36, 37, 148, 149, 239, 240, 241, 242, 321], dtype='int64')

In [522]:
preserve_df = batch_df[batch_df[0] < 1]
preserve_array = preserve_df.index
preserve_array

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            350, 351, 352, 353, 354, 355, 356, 357, 358, 359],
           dtype='int64', length=350)

In [523]:
len(preserve_array)

350

In [524]:
len(wasted_array)

10

In [525]:
dataset[35*60:(35*60)+60]

,time,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,...,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
2100,2020-07-07 15:35:00,0.10178,1.45557,403.78854,985.37353,32.59527,76.15913,2842.39282,36.47974,77.36610,...,18728.0,356.08362,348.41577,0,10077.0,27627.0,0,0,0,0
2101,2020-07-07 15:35:01,0.10178,1.45786,403.78854,985.37353,32.59527,76.11606,2838.04102,36.48547,77.28599,...,18732.0,354.74536,348.66901,0,10079.0,27627.0,0,0,0,0
2102,2020-07-07 15:35:02,0.10178,1.45679,403.78854,985.37353,32.59527,76.09527,2826.10840,36.48280,77.06074,...,18757.0,355.52301,349.46472,0,10074.0,27627.0,0,0,0,0
2103,2020-07-07 15:35:03,0.10178,1.45618,403.78854,985.37353,32.59527,76.02886,2822.59912,36.48127,76.88635,...,18811.0,356.57190,350.11572,0,10081.0,27627.0,0,0,0,0
2104,2020-07-07 15:35:04,0.10178,1.45992,403.78854,985.37353,32.59527,76.02498,2825.68750,36.49062,76.77198,...,18845.0,357.81976,350.82105,0,10079.0,27627.0,0,0,0,0
2105,2020-07-07 15:35:05,0.10178,1.46365,403.78854,985.37353,32.59527,76.02109,2828.77588,36.49996,76.65761,...,18909.0,357.54846,351.32739,0,10082.0,27627.0,0,0,0,0
2106,2020-07-07 15:35:06,0.10178,1.46434,403.78854,985.37353,32.59527,75.98675,2829.86383,36.50168,76.50637,...,18968.0,358.09100,352.55713,0,10083.0,27627.0,0,0,0,0
2107,2020-07-07 15:35:07,0.10178,1.46503,403.78854,985.37353,32.59527,75.95240,2830.95178,36.50339,76.35513,...,18988.0,359.84521,352.75610,0,10077.0,27627.0,0,0,0,0
2108,2020-07-07 15:35:08,0.10178,1.47342,403.78854,985.37353,32.59527,75.92616,2831.72388,36.52438,75.79694,...,18967.0,356.57190,352.91888,0,10081.0,27627.0,0,0,0,0
2109,2020-07-07 15:35:09,0.10178,1.47235,403.78854,985.37353,32.59527,75.89323,2833.12744,36.52171,76.02171,...,19007.0,358.19952,353.56988,0,10078.0,27627.0,0,0,0,0


In [526]:
used_dataset = dataset
new_dataset = dataset2

In [527]:
discard_array = None
for batch_index in wasted_array:
    print(batch_index)
    if type(discard_array) != "numpy.ndarray":
        index = batch_index * 60 
        used_dataset[index:(index+60)] = new_dataset[index:(index+60)]
       

35
36
37
148
149
239
240
241
242
321


In [528]:
used_dataset[35*60:(35*60)+60]

,time,P1_B2004,P1_B2016,P1_B3004,P1_B3005,P1_B4002,P1_B4005,P1_B400B,P1_B4022,P1_FCV01D,...,P4_ST_GOV,P4_ST_LD,P4_ST_PO,P4_ST_PS,P4_ST_PT01,P4_ST_TT01,attack,attack_P1,attack_P2,attack_P3
2100,2020-07-09 15:35:00,0.09874,1.31353,382.21698,1118.61084,32.0,100.0,2836.21582,35.53696,100.0,...,16679.0,295.35590,309.06396,0,10052.0,27554.0,0,0,0,0
2101,2020-07-09 15:35:01,0.09874,1.31185,382.21698,1118.61084,32.0,100.0,2833.26807,35.53276,100.0,...,16603.0,295.62720,307.92462,0,10052.0,27560.0,0,0,0,0
2102,2020-07-09 15:35:02,0.09874,1.31063,382.21698,1118.61084,32.0,100.0,2842.53296,35.52971,100.0,...,16534.0,294.57825,307.11084,0,10052.0,27557.0,0,0,0,0
2103,2020-07-09 15:35:03,0.09874,1.30483,382.21698,1118.61084,32.0,100.0,2838.60254,35.51521,100.0,...,16500.0,292.93256,306.00769,0,10052.0,27553.0,0,0,0,0
2104,2020-07-09 15:35:04,0.09874,1.30010,382.21698,1118.61084,32.0,100.0,2829.33740,35.50339,100.0,...,16447.0,294.10809,304.99493,0,10052.0,27557.0,0,0,0,0
2105,2020-07-09 15:35:05,0.09874,1.29308,382.21698,1118.61084,32.0,100.0,2836.63721,35.48584,100.0,...,16357.0,292.15497,303.42157,0,10052.0,27551.0,0,0,0,0
2106,2020-07-09 15:35:06,0.09874,1.29125,382.21698,1118.61084,32.0,100.0,2835.51392,35.48126,100.0,...,16293.0,290.81671,302.42694,0,10052.0,27553.0,0,0,0,0
2107,2020-07-09 15:35:07,0.09874,1.28835,382.21698,1118.61084,32.0,100.0,2843.23486,35.47401,100.0,...,16296.0,290.09332,302.48120,0,10052.0,27557.0,0,0,0,0
2108,2020-07-09 15:35:08,0.09874,1.28499,382.21698,1118.61084,32.0,100.0,2831.30273,35.46562,100.0,...,16252.0,290.50928,301.25146,0,10052.0,27556.0,0,0,0,0
2109,2020-07-09 15:35:09,0.09874,1.27950,382.21698,1118.61084,32.0,100.0,2844.63892,35.45189,100.0,...,16276.0,294.75909,301.45038,0,10052.0,27562.0,0,0,0,0


In [529]:
len(used_dataset)

43201

In [530]:
X_train_full, X_valid_full, y_train, y_valid = data_preprocessing(used_dataset)
X_train, y_train, X_valid, y_valid, train_dataloader = getDataloader(X_train_full,y_train,X_valid_full,y_valid)

In [531]:
import time

time_start=time.time()

epochs = 1

for epoch in range(epochs):
    model.train()
    for batch, (batch_x, batch_y) in enumerate(train_dataloader):   
        batch_x = batch_x.cuda()   
        output,pre_out = model(batch_x)
        output = torch.reshape(output,[-1,1])

        batch_y = np.array(batch_y)
        batch_y = torch.tensor(np.reshape(batch_y,[-1,1]))
        batch_y = batch_y.float()
        batch_y = batch_y.cuda()

        loss = loss_function(output,batch_y)
        acc = evaluate_accuracy(batch_x,batch_y,model)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        batch_array[batch] = loss.item()
        batch_index_array[batch] = batch
        if epoch % 5 == 0:
            print("epoch:{} batch:{} loss:{} acc:{}".format(epoch,batch,loss.item(),acc))

time_end=time.time()
print('totally cost',time_end-time_start)

epoch:0 batch:0 loss:0.036297839134931564 acc:1.0
epoch:0 batch:1 loss:0.03379866108298302 acc:1.0
epoch:0 batch:2 loss:0.03535168245434761 acc:1.0
epoch:0 batch:3 loss:0.03300252929329872 acc:1.0
epoch:0 batch:4 loss:0.03148370236158371 acc:1.0
epoch:0 batch:5 loss:0.03407471254467964 acc:1.0
epoch:0 batch:6 loss:0.029320186004042625 acc:1.0
epoch:0 batch:7 loss:0.027445929124951363 acc:1.0
epoch:0 batch:8 loss:0.03405113145709038 acc:1.0
epoch:0 batch:9 loss:0.03405236080288887 acc:1.0
epoch:0 batch:10 loss:0.03520352765917778 acc:1.0
epoch:0 batch:11 loss:0.028604615479707718 acc:1.0
epoch:0 batch:12 loss:0.02908487059175968 acc:1.0
epoch:0 batch:13 loss:0.02926582843065262 acc:1.0
epoch:0 batch:14 loss:0.028680143877863884 acc:1.0
epoch:0 batch:15 loss:0.029357625171542168 acc:1.0
epoch:0 batch:16 loss:0.03496609255671501 acc:1.0
epoch:0 batch:17 loss:0.03677733615040779 acc:1.0
epoch:0 batch:18 loss:0.029525961726903915 acc:1.0
epoch:0 batch:19 loss:0.03621038421988487 acc:1.0
epo

epoch:0 batch:172 loss:0.015140264295041561 acc:1.0
epoch:0 batch:173 loss:0.022232891991734505 acc:1.0
epoch:0 batch:174 loss:0.019579147920012474 acc:1.0
epoch:0 batch:175 loss:0.01748916320502758 acc:1.0
epoch:0 batch:176 loss:0.021850917488336563 acc:1.0
epoch:0 batch:177 loss:0.017463818192481995 acc:1.0
epoch:0 batch:178 loss:0.01762232556939125 acc:1.0
epoch:0 batch:179 loss:0.01620330475270748 acc:1.0
epoch:0 batch:180 loss:0.01941581815481186 acc:1.0
epoch:0 batch:181 loss:0.019373200833797455 acc:1.0
epoch:0 batch:182 loss:0.016248153522610664 acc:1.0
epoch:0 batch:183 loss:0.017620114609599113 acc:1.0
epoch:0 batch:184 loss:0.017903128638863564 acc:1.0
epoch:0 batch:185 loss:0.016896188259124756 acc:1.0
epoch:0 batch:186 loss:0.016704488545656204 acc:1.0
epoch:0 batch:187 loss:0.018095986917614937 acc:1.0
epoch:0 batch:188 loss:0.015902044251561165 acc:1.0
epoch:0 batch:189 loss:0.01669822819530964 acc:1.0
epoch:0 batch:190 loss:0.016119614243507385 acc:1.0
epoch:0 batch:191

epoch:0 batch:337 loss:0.013277493417263031 acc:1.0
epoch:0 batch:338 loss:0.015049231238663197 acc:1.0
epoch:0 batch:339 loss:0.011504732072353363 acc:1.0
epoch:0 batch:340 loss:0.012824506498873234 acc:1.0
epoch:0 batch:341 loss:0.011930469423532486 acc:1.0
epoch:0 batch:342 loss:0.011546380817890167 acc:1.0
epoch:0 batch:343 loss:0.012659518979489803 acc:1.0
epoch:0 batch:344 loss:0.012329697608947754 acc:1.0
epoch:0 batch:345 loss:0.013101286254823208 acc:1.0
epoch:0 batch:346 loss:0.010489082895219326 acc:1.0
epoch:0 batch:347 loss:0.012351990677416325 acc:1.0
epoch:0 batch:348 loss:0.010617952793836594 acc:1.0
epoch:0 batch:349 loss:0.011843107640743256 acc:1.0
epoch:0 batch:350 loss:0.012041093781590462 acc:1.0
epoch:0 batch:351 loss:0.012873681262135506 acc:1.0
epoch:0 batch:352 loss:0.012399543076753616 acc:1.0
epoch:0 batch:353 loss:0.01005115732550621 acc:1.0
epoch:0 batch:354 loss:0.010302446782588959 acc:1.0
epoch:0 batch:355 loss:0.013327398337423801 acc:1.0
epoch:0 batch

IndexError: index 360 is out of bounds for axis 0 with size 360

In [540]:
model.eval()
x = torch.tensor( X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
y = torch.tensor(np.reshape(y,[-1,1]))
y = y.float()
    
x = x.cuda()
y = y.cuda()

x_valid = torch.tensor(X_valid[:,:,:]).float()
x_valid = torch.where(torch.isnan(x_valid), torch.full_like(x_valid, 0), x_valid)
    
y_val = y_valid[:]
y_val = np.array(y_val)
y_val = torch.tensor(np.reshape(y_val,[-1,1]))
y_val = y_val.float()

x_valid = x_valid.cuda()
y_val = y_val.cuda()

print("epoch:{} loss:{}".format(epoch,loss.item()))
print("train acc", evaluate_accuracy(x,y,model))
print("valid acc:",evaluate_accuracy(x_valid,y_val,model))

epoch:0 loss:0.012657048180699348
train acc 0.9960936369686623
valid acc: 1.0


In [532]:
len(X_train)

34559

In [533]:
train_dataloader.batch_size

60

In [534]:
type(remind_array)

numpy.ndarray

In [535]:
remind_array = X_train[0:0+60]
type(remind_array)

numpy.ndarray

In [536]:
remind_array = None
for batch_index in preserve_array:
    print(batch_index)
    if type(remind_array) != "numpy.ndarray":
        remind_array = X_train[batch_index:batch_index+60]
    batch_index = index * 60    
    remind_array = np.r_[remind_array,X_train[batch_index:batch_index+60]]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285

In [537]:
len(remind_array)

60

In [538]:
4740 / 60

79.0

In [539]:
selected_df.index

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            350, 351, 352, 353, 354, 355, 356, 357, 358, 359],
           dtype='int64', length=350)

 78377   278.5211410522461
 8936    33.402079820632935
 

In [ ]:
len(X_train)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
# import time
# print(len(X_train))
# time_start=time.time()
# epochs = 200

# for epoch in range(epochs):
#     model_2.train()
#     x = torch.tensor( X_train[:,:,:]).float()
#     x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
#     y = y_train[:]
#     y = np.array(y)
#     y = torch.tensor(np.reshape(y,[-1,1]))
#     y = y.float()
    
#     x = x.cuda()
#     y = y.cuda()
    
#     output,pre_out = model_2(x)
#     output = torch.reshape(output,[-1,1])
#     loss = loss_function(output,y)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    
#     if epoch % 5 == 0:
#         print("epoch:{} loss:{}".format(epoch,loss.item()))
        
# time_end=time.time()
# print('totally cost',time_end-time_start)     
        

4019 42.43043494224548 40199  281.12068939208984 120600 820.6956114768982  201001 1340  361803 2563.249900817871

In [ ]:
# model_2.eval()
# x = torch.tensor( X_train[:,:,:]).float()
# x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
# y = y_train[:]
# y = np.array(y)
# y = torch.tensor(np.reshape(y,[-1,1]))
# y = y.float()
    
# x = x.cuda()
# y = y.cuda()

# x_valid = torch.tensor(X_valid[:,:,:]).float()
# x_valid = torch.where(torch.isnan(x_valid), torch.full_like(x_valid, 0), x_valid)
    
# y_val = y_valid[:]
# y_val = np.array(y_val)
# y_val = torch.tensor(np.reshape(y_val,[-1,1]))
# y_val = y_val.float()

# x_valid = x_valid.cuda()
# y_val = y_val.cuda()

# print("epoch:{} loss:{}".format(epoch,loss.item()))
# print("train acc", evaluate_accuracy(x,y,model_2))
# print("valid acc:",evaluate_accuracy(x_valid,y_val,model_2))

In [ ]:
# import time

# time_start=time.time()
# epochs = 200

# for epoch in range(epochs):
#     model.train()
#     x = torch.tensor( X_train[:,:,:]).float()
#     x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
#     y = y_train[:]
#     y = np.array(y)
#     y = torch.tensor(np.reshape(y,[-1,1]))
#     y = y.float()
    
#     x = x.cuda()
#     y = y.cuda()
    
#     output,pre_out = model(x)
#     output = torch.reshape(output,[-1,1])
#     loss = loss_function(output,y)
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    
#     if epoch % 5 == 0:
#         print("epoch:{} loss:{}".format(epoch,loss.item()))
        
# time_end=time.time()
# print('totally cost',time_end-time_start)     
        

In [ ]:
# model.eval()
# x = torch.tensor( X_train[:,:,:]).float()
# x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
# y = y_train[:]
# y = np.array(y)
# y = torch.tensor(np.reshape(y,[-1,1]))
# y = y.float()
    
# x = x.cuda()
# y = y.cuda()

# x_valid = torch.tensor(X_valid[:,:,:]).float()
# x_valid = torch.where(torch.isnan(x_valid), torch.full_like(x_valid, 0), x_valid)
    
# y_val = y_valid[:]
# y_val = np.array(y_val)
# y_val = torch.tensor(np.reshape(y_val,[-1,1]))
# y_val = y_val.float()

# x_valid = x_valid.cuda()
# y_val = y_val.cuda()

# valid_acc = evaluate_accuracy(x_valid,y_val,model_2)
# print("epoch:{} loss:{}".format(epoch,loss.item()))
# print("train acc", evaluate_accuracy(x,y,model))
# print("valid acc:",evaluate_accuracy(x_valid,y_val,model))

9031    64.93641090393066
23512   162.0211820602417
39187   271.4542453289032
70538   484.0234043598175


In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
#4019 42.43043494224548 40199  40199  281.12068939208984 120600 820.6956114768982  201001 1340  361803 2563.249900817871
x = [[4019],[120600],[201001] ]
y = [[42],[820.6956114768982],[1340]]
reg = LinearRegression().fit(x,y)
x_pre = np.linspace(0,370000,5)
x_pre = x_pre.reshape(-1,1)
y_pre = reg.predict(x_pre)
plt.plot(x, y,'o')
plt.plot(x_pre, y_pre)
plt.show()

In [ ]:
reg.predict([[361803]])

In [ ]:
a = reg.coef_
b = reg.intercept_
print(a,b)

In [ ]:
model.eval()
x = torch.tensor( X_train[:,:,:]).float()
x = torch.where(torch.isnan(x), torch.full_like(x, 0), x)
    
y = y_train[:]
y = np.array(y)
y = torch.tensor(np.reshape(y,[-1,1]))
y = y.float()
    
x = x.cuda()
y = y.cuda()

x_valid = torch.tensor(X_valid[:,:,:]).float()
x_valid = torch.where(torch.isnan(x_valid), torch.full_like(x_valid, 0), x_valid)
    
y_val = y_valid[:]
y_val = np.array(y_val)
y_val = torch.tensor(np.reshape(y_val,[-1,1]))
y_val = y_val.float()

x_valid = x_valid.cuda()
y_val = y_val.cuda()

train_acc = evaluate_accuracy(x_valid,y_val,model)
print("epoch:{} loss:{}".format(epoch,loss.item()))
print("train acc", evaluate_accuracy(x,y,model))
print("valid acc:",train_acc)

In [ ]:
print(y_valid[:,0])
print(output_cpu)

In [ ]:
import sklearn.metrics as sm
output,pre_out = model(x_valid)
output = torch.reshape(output,[-1,1])
output = output.ge(0.5)

output_cpu = output.cpu()
cr = sm.classification_report(y_valid[:,0], output_cpu)
cr

# update